In [10]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from Autoencoder import ConvAutoencoder as convenc 
from DataGenerator import DatasetGenerator
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import pickle
from torch.autograd import Variable

In [33]:

path="/content/drive/My Drive/dataset/"
model_file="/content/drive/My Drive/final.pkl"

transform = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.Resize((224,224)),
    transforms.ToTensor()
    
])

train_dataset = DatasetGenerator(pathImageDirectory=path, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=1, 
                                           shuffle=False, num_workers=8)



In [3]:
# Training the Autoencoder

model=convenc()
# specify loss function
criterion = nn.BCELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 100

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data in train_loader:
        # _ stands in for filename, here
        # no need to flatten images
        images, _ = data
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(images)
        # calculate the loss
        loss = criterion(outputs, images)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))
torch.save(model.state_dict(), model_file) #saving the model

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 	Training Loss: 3.632466
Epoch: 2 	Training Loss: 3.430237
Epoch: 3 	Training Loss: 3.421157
Epoch: 4 	Training Loss: 3.410888
Epoch: 5 	Training Loss: 3.407243
Epoch: 6 	Training Loss: 3.404159
Epoch: 7 	Training Loss: 3.405836
Epoch: 8 	Training Loss: 3.401446
Epoch: 9 	Training Loss: 3.398049
Epoch: 10 	Training Loss: 3.400801
Epoch: 11 	Training Loss: 3.395616
Epoch: 12 	Training Loss: 3.397542
Epoch: 13 	Training Loss: 3.398632
Epoch: 14 	Training Loss: 3.398113
Epoch: 15 	Training Loss: 3.394994
Epoch: 16 	Training Loss: 3.395583
Epoch: 17 	Training Loss: 3.397152
Epoch: 18 	Training Loss: 3.396963
Epoch: 19 	Training Loss: 3.395959
Epoch: 20 	Training Loss: 3.396124
Epoch: 21 	Training Loss: 3.394416
Epoch: 22 	Training Loss: 3.394082
Epoch: 23 	Training Loss: 3.394741
Epoch: 24 	Training Loss: 3.395676
Epoch: 25 	Training Loss: 3.394509
Epoch: 26 	Training Loss: 3.389257
Epoch: 27 	Training Loss: 3.393067
Epoch: 28 	Training Loss: 3.391237
Epoch: 29 	Training Loss: 3.3

In [34]:
### creating input for KNN model

class Identity(nn.Module): ##this class is used to get encoder output out of the autoencoder model by converting decoder layer to identity 
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

model=convenc().cuda()


model.load_state_dict(torch.load(model_file))

model.t_conv1=Identity()
model.t_conv2=Identity()
model.t_conv3=Identity()


train_flatten=np.empty((0,3136)) #3136 after flattening the output of model product of C,h,w is 3136
print(train_flatten.shape)
i=0
filenames=[]
for files,filename in train_loader:

  images=Variable(files.cuda())
  output = model(images)
  
  temp=torch.flatten(output) 
  
  temp=temp.to('cpu').detach().numpy()
  train_flatten=np.append(train_flatten,np.array([temp]),axis=0)
  filenames.append(filename)
  

print(train_flatten.shape)  
print(len(filenames))

'''
Uncomment below to save the filenames array as the index of this array will be used to match the 
index from KNN to retrieve image file name
'''

# from numpy import save
# arr_filename=np.array(filenames)
# save('/content/drive/My Drive/filename.npy',arr_filename)


(0, 3136)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(4738, 3136)
4738


In [35]:
# passing encoded images to KNN

knn = NearestNeighbors(n_neighbors=5, metric="cosine")
knn.fit(train_flatten)

# test KNN model

for i, emb_flatten in enumerate(train_flatten):
    _, indices = knn.kneighbors([emb_flatten]) # find k nearest train neighbours
    
    print(indices)

### Uncomment below to save the KNN model

# with open(''/content/drive/My Drive/KNN.pkl'', 'wb') as f:
#    pickle.dump(knn, f)

Performing image retrieval on test images...
[[   0   12 4166 4570 3262]]
